In [42]:
from pdfminer import pdfpage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
import nltk
from nltk import sent_tokenize, word_tokenize
import pandas as pd
from io import StringIO
import csv

from flair.tokenization import SegtokTokenizer
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle
import re

In [74]:
def pdf2token(pdf,loc):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams(char_margin=30, line_margin=2, boxes_flow=1)
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    caching = True
    pagenos = set()
    fp = open(pdf, 'rb')
    writer = csv.writer(open(loc, 'w', newline='', encoding="utf-8"))

    for pagenumber, page in enumerate(pdfpage.PDFPage.get_pages(fp, check_extractable=True)):
        #print(pagenumber)
        if pagenumber:
            interpreter.process_page(page)
            data = retstr.getvalue()
            encoded_string = data.encode("ascii", "ignore")
            data = encoded_string.decode()
            #print('PAGE DATA +++++++++++++' + data)
            #sent_text = sent_tokenize(data) # NLTK sentence tokenizer (poor results)
            splitter = SegtokSentenceSplitter() # Flair sentence spliter + word tokenizer
            sent_text = splitter.split(data)
            for sent in sent_text:
                #print(f'pre: {sent}')       
                #print(f'ASCII removed : {cleansent} ')
                #cleansent = clean(sent)
                #cleansent = " ".join(cleansent.split()) # Remove extra whitespaces
                #print(f'post: {cleansent} ')
                #writer.writerow([cleansent.lower()])
                #writer.writerow('\n')
                writer.writerow([sent.to_tokenized_string().lower()])

def to_plain_sent(tokens): # Convert tokenized(cleaned) words back to sentence
    
    sent = " ".join([t.text for t in tokens])
    return sent.rstrip()

def txttrain(train):
    entpair = []
    eg = []
    ent = []
    with open(train, 'r', encoding="utf-8") as f:
        for line in f: 
            text = line.rstrip('\n')
            defi = []
            defi = text.lower().split(',')
            defi[0] = defi[0].replace(' ','I')
            if len(defi)>=3:
                for w in range(1,len(defi)-1):
#                    print(str(w))
                    defi[1] = defi[1]+', '+defi[w+1]
            defi[1] = defi[1].strip('\n "()')
            tokeniser = SegtokTokenizer() # Flair sentence tokenizer
            sent_text = tokeniser.tokenize(defi[1])
            new = to_plain_sent(sent_text)
#            print(f'Tok: {new} Old: {defi[1]}')
            temp = new
            #defi[1] = clean(defi[1])
            #defi[1] = " ".join(defi[1].split())
            #temp = defi[1]
            defi[1] = defi[0]
            defi[0] = temp
            print('[ '+defi[0]+' , '+defi[1]+' ]')
            eg.append(defi[0])
            ent.append(defi[1])
        entpair = list(zip(eg,ent))
    return(entpair)
            

    
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip("[]'")
    string = string.strip("[]'")
    match = []
    '''
    i=0
    x = 0
    while x!= -1:
      x = string.find(pattern, i)
      if x!= -1:           
        tup = (x, x+len(pattern))
        #print('Value: '+pattern+'Location:    '+str(tup)+'\n\n'+string)
        match.append(tup)
        i = x+len(pattern)
    #match.pop()
    return match, string
    '''
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        #print(string)
        match_list.append(match_tup)
    return match_list, string
    
        
    

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    tokeniser = SegtokTokenizer() # Flair sentence tokenizer

    s = s.strip("[]'")
    #print(s)
    
    word_dict = []
    first = 0    
    itr = 0
    
    for multiple in match_list:
        beg = False 
        #print('current itteration- '+str(multiple))
        start = multiple[0]
        end = multiple[1]
        e_type = multiple[2]
        temp_str = s[start:end]
        tmp_list = tokeniser.tokenize(temp_str) #word_tokenize(temp_str)
        if itr!=(len(match_list)-1):
            substring = s[first:end]
            #print('Itteration: '+str(itr)+str(substring))
        else:
            substring = s[first:]
            #print('Itteration: '+str(itr)+str(substring))
            
        print('First Identifier:    '+tmp_list[0].text) 
        tokens = tokeniser.tokenize(substring)
        for word in tokens:
            x = word.text.find(tmp_list[0].text)
            if x!=0:
                word_dict.append(word.text+' '+'O')
            if x==0 and beg is False:    
                word_dict.append(word.text+' '+'B-' + e_type)
                beg = True
                print('Begiening detected : '+word.text)
                continue
                
            if len(tmp_list) > 1 and beg is True:
                #print('We in the long sent: :: : : : : : : ')
                #print(f'Rest KEY = {str(tmp_list[1:])}')
                for w in tmp_list[1:]:
                    t = word.text.find(w.text)
                    if t==0:
                        #print(str(t)+' Inner detected : '+word+' idf> '+w)
                        word_dict.pop()
                        word_dict.append(word.text+' '+'I-' + e_type)
        first=end
        itr=itr+1
    #print('Final Word outcome:::    '+str(word_dict)+'\n\n\n')
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "(", ")", "/", "*", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", ",","."]
    text = text.strip('\n ')
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\br', ' ')
    for i in text:
        i = i.replace("'", ' ').replace("`", ' ').replace("°",'')
        if i in filters:
            text = text.replace(i, " "+i+' ')
    text = " ".join(text.split())
    return text

def create_data(df, text, filepath):
    '''
    The function responsible for the creation of data in the said format.
    1234
    1>1,a>
    '''
    appended = r'H:/Code/Doc_IMG-OCR/trainer/ner/training/train.txt'
    with open(filepath , 'a', newline='', encoding="utf-8") as f:
        for sentence in text:
            text_ = sentence       
            match_list = []
            for i,y in df:
                a, text_ = matcher(str(sentence), i)
                if a:
                    #print(i)
                    #print(str(a)+str(text_))
                    for itr in a:
#                        print((itr[0], itr[1], y))
                        match_list.append((itr[0], itr[1], y))
            if match_list:
#                print(f'{str(len(match_list))} Sending detected sentence')
                print('Final match list:    '+str(match_list))
                d = mark_sentence(str(sentence), match_list)
                for i in d:
                    f.writelines(str(i)+'\n')
                f.writelines('\n')

In [76]:
def main():
    names = ['MSEDCL','BHEL','RGGVY','ONGC','ADANI' ,'AGRA' ,'BSES', 'DUGJY' ,'KEONJ' ,'Reliance' ,'TATA' ,'TSLT', 'APDCL', 'KVX', 'PVVNL']
    for doc in tqdm(names,ncols=150,
                desc=f" Processing Files. . . . "):
        print(f'File currently being worked on: {doc}')
        filename = doc #ADANI AGRA BSES DUGJY KEONJ PGCIL Reliance TATA TSLT APDCL KVX PVVNL
        ## path to save the txt file.
        pdf = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/'+filename +'.pdf'
        filepath = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/raw/'+filename +'.txt'
        sentoken = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/training/'+filename +'.csv'
        final = r'C:/Users/33669/PycharmProjects/OCR/trainer/ner_data/train.txt'
        
        ## creating the file.
        sentences = []
        pdf2token(pdf,sentoken)
        data = txttrain(filepath) #Changed Drum info in txt file revert it back    
        
        with open(sentoken, 'r', encoding="utf-8") as read_obj:
            csv_reader = csv.reader(read_obj)
            for row in csv_reader:
                sentences.append(row)            
        create_data(data, sentences, final)
        
        with open(sentoken, 'r', encoding="utf-8") as read_obj:
            csv_reader = csv.reader(read_obj)
            for row in csv_reader:
                sentences.append(row)            
        create_data(data, sentences, final)

if __name__ == '__main__':
    main()


 Processing Files. . . . :   0%|                                                                                               | 0/15 [00:00<?, ?it/s]

File currently being worked on: MSEDCL
[ stranded compacted h2 / h4 grade aluminium , conductor ]
[ semi-conducting cross link material , conductorIscreen ]
[ xlpe , insulation ]
[ extruded semi-conducting cross link material , insulationIscreen ]
[ metallic layer of copper tapes . , insulationIscreen ]
[ heat resistant pvc , innerIsheath ]
[ single galvanized round steel wires , armour ]
[ pvc compound , outerIsheath ]
[ black , outerIsheath ]
[ non erasable sequential marking of length shall be provided by embossing , marking ]
[ manufactures name and / or trade name . , marking ]
[ voltage grade . , marking ]
[ year of manufacture . , marking ]
[ m.s.e.d.c.l . , marking ]
[ successive length . , marking ]
[ size of cable , marking ]
[ isi mark , marking ]
[ 250 running meters with plus minus 5 % tolerance , packing ]
[ non returnable wooden drum , packing ]
Final match list:    [(42, 46, 'insulation')]
First Identifier:    xlpe
Begiening detected : xlpe
Final match list:    [(42, 46

 Processing Files. . . . :   7%|█████▊                                                                                 | 1/15 [00:03<00:43,  3.10s/it]

First Identifier:    xlpe
Begiening detected : xlpe
Final match list:    [(136, 140, 'insulation')]
First Identifier:    xlpe
Begiening detected : xlpe
Final match list:    [(135, 177, 'conductor'), (262, 297, 'conductorIscreen'), (328, 332, 'insulation'), (253, 297, 'insulationIscreen')]
First Identifier:    stranded
Begiening detected : stranded
First Identifier:    semi-conducting
Begiening detected : semi-conducting
First Identifier:    xlpe
Begiening detected : xlpe
First Identifier:    extruded
Final match list:    [(75, 110, 'conductorIscreen'), (17, 21, 'insulation'), (66, 110, 'insulationIscreen'), (133, 165, 'insulationIscreen')]
First Identifier:    semi-conducting
Begiening detected : semi-conducting
First Identifier:    xlpe
First Identifier:    extruded
Begiening detected : extruded
First Identifier:    metallic
Begiening detected : metallic
Final match list:    [(149, 167, 'innerIsheath'), (382, 417, 'armour'), (595, 607, 'outerIsheath')]
First Identifier:    heat
Begien

 Processing Files. . . . :  13%|███████████▏                                                                        | 2/15 [24:23<3:06:22, 860.19s/it]

...

File currently being worked on: KVX
[ 33 kv 3x300mm2 xlpe armoured cable , cableItype ]
[ aluminum , conductor ]
[ semi-conducting , conductorIscreen ]
[ xlpe , insulation ]
[ semi-conducting , insulationIscreen ]
[ type st-2 pvc , innerIsheath ]
[ galvanized steel strip , armour ]
[ type st-2 pvc , outerIsheath ]
[ rodent and termite , outerIsheath ]
[ - 5 ) minimum ambient air temperature ( oc , temperature ]
[ 250 m. non-standard length , length ]
[ manufacturers name and address , marking ]
[ size of cable , marking ]
[ isi marking , marking ]
[ specification no , marking ]
[ voltage grade and size of cores , marking ]
[ cmrl certification no , marking ]
[ year of manufacturing , marking ]
[ non returnable wooden drum , packing ]
Final match list:    [(98, 132, 'cableItype'), (113, 117, 'insulation')]
First Identifier:    33
Begiening detected : 33
First Identifier:    xlpe
Final match list:    [(0, 4, 'insulation')]
First Identifier:    xlpe
Begiening detected : xlpe
Final match l

 Processing Files. . . . :  93%|████████████████████████████████████████████████████████████████████████████████▎     | 14/15 [31:39<00:23, 23.43s/it]

Begiening detected : aluminum
Final match list:    [(62, 66, 'insulation'), (80, 102, 'armour')]
First Identifier:    xlpe
Begiening detected : xlpe
First Identifier:    galvanized
Begiening detected : galvanized
File currently being worked on: PVVNL
[ 11 kv , 3x300 sq. mm xlpe ( armoured ) cable , cableItype ]
[ aluminum conductor , conductor ]
[ semi - conducting screen , conductorIscreen ]
[ xlpe insulation , insulation ]
[ bonded type and strippable on adequate heating , insulationIscreen ]
[ copper tape , insulationIscreen ]
[ pvc inner sheath , innerIsheath ]
[ galvanized steel strip , armour ]
[ dimensions of steel strip shall be as per table 4 of is : 7098 ( part-ii )/ 1985 , armour ]
[ extruded pvc outer sheath of type st-2 , outerIsheath ]
[ pvc fillers , filler ]
[ weight of aluminium conductor used in the cable shall be 2433 kg / km , conductorIweight ]
[ 250 mts , packing ]
[ non-returnable wooden drums , packing ]
[ property of pvvnl , marking ]
[ name of manufacturer , m

 Processing Files. . . . : 100%|█████████████████████████████████████████████████████████████████████████████████████| 15/15 [31:40<00:00, 126.71s/it]

First Identifier:    250
Begiening detected : 250
Final match list:    [(55, 72, 'marking'), (80, 100, 'marking'), (109, 128, 'marking'), (160, 173, 'marking'), (178, 227, 'marking')]
First Identifier:    property
Begiening detected : property
First Identifier:    name
Begiening detected : name
First Identifier:    year
Begiening detected : year
First Identifier:    voltage
Begiening detected : voltage
First Identifier:    size
Begiening detected : size
Final match list:    [(48, 75, 'packing')]
First Identifier:    non-returnable
Begiening detected : non-returnable
Final match list:    [(7, 27, 'marking')]
First Identifier:    name
Begiening detected : name
